In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import KerasTools as KT
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
from keras.datasets import mnist
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical

In [ ]:
# Load MNIST data and preprocess it
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Generate a neural network
#
# We will use a sequential model (one layer after another)
# The network will accept as input flattened and normalized images of 1D size 784. 
# Two intermediate layers with 64 neurons and ReLU ('rectified linear unit') activation
# The output layer will have 10 neurons and Softmax activation
#
# We will use 'categorical_crossentropy' for the loss function
# The optimizer will be SGD ('stochastic gradient descent') with a learning rate of 0.01
# We will additionally measure the predictive accuracy

def build_network():
    network = models.Sequential()
    network.add(layers.Dense(64, activation='relu', input_shape=(28*28*1,)))
    network.add(layers.Dense(64, activation='relu'))
    network.add(layers.Dense(10, activation='softmax'))

    network.compile(optimizer=optimizers.sgd(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
# Build a fresh new neural network and plot its architecture
network = build_network()
network.summary()

In [ ]:
# Train the network with the MNIST training set for 5 epochs and a batch size of 128
# Depending on your hardware, training can take a while. Training progress is plotted for each epoch.
# The fit() function returns the training loss/accuracy history of each epoch.

epochs = 5
history = network.fit(train_images, train_labels, epochs=epochs, batch_size=128)

In [ ]:
# After training, evaluate the generalizing power of the network with the test set. 
# How does it fare on data it hasn't seen before?

test_loss, test_acc = network.evaluate(test_images, test_labels)
print()
print("Test loss: {}".format(test_loss))
print("Test accuracy: {}".format(test_acc))
history.history['test_loss'] = test_loss
history.history['test_acc'] = test_acc
history.history['epochs'] = epochs

In [ ]:
# Plot loss/accuracy training history and test results in some nice graphs
KT.plot_history(history.history)

In [ ]:
# Save the network for later production use
network.save("./mnist_trained.h5")